In [18]:
import pandas as pd
import psycopg2
import time

# Connect to postgres
dbname = "weather"
user = "root"
password = "root"
host = "localhost"
port = "5401"

In [19]:
file_path = '../../dataset-Climate/GlobalYearlyLandTempByState.csv'
year_range = range(1960, 2014)
country_col_name = 'Country Code'
debug_print = True

In [20]:
def custom_print(*args, **kwargs):
    if debug_print:
        print(*args, **kwargs)

In [21]:
def check_null(amount):
    # Handle NaN values and set amount to NULL
    return None if pd.isna(amount) or pd.isnull(amount) else amount

In [22]:
yearly_temp_df = pd.read_csv(file_path)
yearly_temp_df.head(1)

,Year,AverageTemperature,MinimumTemperature,MaximumTemperature,State,Country
0,1750,NaN,NaN,NaN,Adygey,Russia


In [23]:
with psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
) as connection:
    cursor = connection.cursor()
    query = f"""
    SELECT * FROM country;
"""
    cursor.execute(query)
    all_country = cursor.fetchall()
    country_dict = {name: code for code, name in all_country}

In [24]:
with psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
) as connection:
    cursor = connection.cursor()
    query = f"""
    SELECT * FROM state;
"""
    cursor.execute(query)
    all_state = cursor.fetchall()
    state_dict = {state_name: {"id": id, "country_name": country_name} for id, state_name, country_name in all_state}

In [25]:
def get_state_id(state_name):
    return state_dict[state_name]["id"]

In [26]:
country_dict

{'Antarctica': 'ATC',
 'Argentina': 'ARG',
 'Cambodia': 'KHM',
 'American Samoa': 'ASM',
 'Mexico': 'MEX',
 'Zambia': 'ZMB',
 'Belize': 'BLZ',
 'Bulgaria': 'BGR',
 'Isle Of Man': 'IMN',
 'Niger': 'NER',
 'Saudi Arabia': 'SAU',
 'Botswana': 'BWA',
 'Tajikistan': 'TJK',
 'Puerto Rico': 'PRI',
 'Equatorial Guinea': 'GNQ',
 'Solomon Islands': 'SLB',
 'Iceland': 'ISL',
 'Mauritania': 'MRT',
 'Burkina Faso': 'BFA',
 'Ireland': 'IRL',
 'Federated States Of Micronesia': 'FSM',
 'Kenya': 'KEN',
 'Canada': 'CAN',
 'Sweden': 'SWE',
 'Croatia': 'HRV',
 'Dominica': 'DMA',
 'Austria': 'AUT',
 'Hungary': 'HUN',
 'Netherlands': 'NLD',
 'Japan': 'JPN',
 'Guatemala': 'GTM',
 'Comoros': 'COM',
 'Slovakia': 'SVK',
 'Gaza Strip': 'PSE',
 'World': 'WLD',
 'Congo (Democratic Republic Of The)': 'COD',
 'Nepal': 'NPL',
 'Tunisia': 'TUN',
 'Guam': 'GUM',
 'Laos': 'LAO',
 'Italy': 'ITA',
 'Saint Kitts And Nevis': 'KNA',
 'Tanzania': 'TZA',
 'Somalia': 'SOM',
 'Egypt': 'EGY',
 'France': 'FRA',
 'Colombia': 'COL',

In [27]:
def get_country_code(country_name):
    return country_dict[country_name]

In [28]:
def check_state_exist(connection, state_name, country_name):
    cursor = connection.cursor()
    country_code = get_country_code(country_name)
    check_query = f"""
        SELECT 1
        FROM state
        WHERE name = %s and country_code = %s
        LIMIT 1;
    """
    # start_time_exec_and_fetchone = time.time()
    cursor.execute(check_query, (state_name, country_code))
    result = cursor.fetchone()
    return result is not None

In [29]:
def add_state_to_country(connection, row):
    cursor = connection.cursor()
    
    state = row["State"]
    country = row["Country"]
    

    if check_state_exist(connection, state, country):
        print("state exist")
        return 0
    insert_query = f"""
            INSERT INTO state (
                name, country_code
            )
            VALUES (
                %s, %s
            );
        """
    cursor.execute(insert_query, (state, get_country_code(country)))
    connection.commit()
    return 1
    

In [30]:
def temp_exists(connection, state_name, year, country_name):
    cursor = connection.cursor()

    country_code = get_country_code(country_name)

    check_query = """
        SELECT 1
        FROM temperature
        WHERE state_id = %s
        AND year = %s
        AND country_code = %s
        
        LIMIT 1;
    """
    cursor.execute(check_query, (get_state_id(state_name), year, country_code))
    result = cursor.fetchone()
    cursor.close()

    return result is not None

In [31]:
def add_state_temp(connection, row):
    cursor = connection.cursor()
    state = row["State"]
    country = row["Country"]
    year = row["Year"]
    average_temp = check_null(row['AverageTemperature'])
    min_temp = check_null(row['MinimumTemperature'])
    max_temp = check_null(row['MaximumTemperature'])

    is_temp_exists = temp_exists(connection, state, year, country)
    state_id = get_state_id(state)
    country_code = get_country_code(country)
    if is_temp_exists:
        print("temp_exist: ", state_id, year, country_code)
        return 0
    
    insert_query = f"""
            INSERT INTO temperature (
                year, state_id, country_code,
                average_temp, min_temp, max_temp
            )
            VALUES (
               %s, %s, %s,
               %s, %s, %s
            );
        """
    
    cursor.execute(insert_query,
                   (year, state_id, country_code,
                    average_temp, min_temp, max_temp))
    return 1
        

In [32]:
# Establish a connection to the PostgreSQL database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

    total_insert = 0
    for index, row in yearly_temp_df.iterrows():
        total_insert += add_state_temp(connection, row)
        # total_insert += add_state_to_country(connection, row)
        
    connection.commit()
    connection.close()

    print(f"Data inserted successfully! Total inserts: {total_insert}")


except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")


Data inserted successfully! Total inserts: 53415


In [33]:
unique_combinations = yearly_temp_df[['State', 'Country']].drop_duplicates()
unique_combinations

,State,Country
0,Adygey,Russia
1,Alabama,United States
2,Arkhangel'Sk,Russia
3,Belgorod,Russia
4,Bryansk,Russia
...,...,...
14639,Western Australia,Australia
15124,Acre,Brazil
15270,Northern Territory,Australia
15294,Queensland,Australia


In [34]:
yearly_temp_df

,Year,AverageTemperature,MinimumTemperature,MaximumTemperature,State,Country
0,1750,NaN,NaN,NaN,Adygey,Russia
1,1750,17.653,6.950,28.464,Alabama,United States
2,1750,-1.366,-15.102,13.995,Arkhangel'Sk,Russia
3,1750,6.974,-7.352,21.723,Belgorod,Russia
4,1750,6.284,-7.521,19.975,Bryansk,Russia
...,...,...,...,...,...,...
53410,2013,5.618,-10.928,18.411,Yaroslavl',Russia
53411,2013,2.445,-24.428,21.207,Yevrey,Russia
53412,2013,-2.819,-22.899,12.458,Yukon,Canada
53413,2013,17.597,9.053,21.860,Yunnan,China
